# 영어 Word2Vec 만들기
- https://wikidocs.net/50739
- 영어로 된 코퍼스를 다운받아 전처리를 수행
- 전처리한 데이터를 바탕으로 Word2Vec 작업을 진행

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

### 훈련 데이터 이해하기

In [ ]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7fdb38879590>)

In [ ]:
!ls -l
#ted_en-20160408.xml 74메가정도됨

total 72792
drwxr-xr-x 1 root root     4096 Feb 22 14:38 sample_data
-rw-r--r-- 1 root root 74533638 Feb 24 00:54 ted_en-20160408.xml


### 훈련 데이터 전처리하기
- 실질적 데이터는 영어문장으로만 구성된 내용을 담고 있는 \<content>와 \</content> 사이의 내용
- (Laughter)나 (Applause)와 같은 배경음을 나타내는 단어도 등장하는데 이 또한 제거

In [ ]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져옴.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

In [ ]:
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

In [ ]:
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

In [ ]:
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

In [ ]:
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]
# ~~~~~~~~~~~~~~~~전처리끝~~~~~~~~~~~~~~~~

In [ ]:
print(f'총 샘플의 개수 : {len(result)}')
for line in result[:3]: #앞3줄만
    print(line)
#총 샘플의 개수 : 273424 ['here', 'are', 'two', 'reasons'.....

총 샘플의 개수 : 273424
['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


### Word2Vec 훈련시키기
- size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
- window = 컨텍스트 윈도우 크기
- min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
- workers = 학습을 위한 프로세스 수
- sg = 0은 CBOW, 1은 Skip-gram

In [ ]:
#코랩에는 이미 gensim 있어요 ><
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0) #100차원 앞뒤단어5개보겠다 빈도수5이하인건안보겠다 CBOW

In [ ]:
model_result = model.wv.most_similar("man") #man과 유사한 단어는? 기본10개뽑아드림
model_result

[('woman', 0.8661956787109375),
 ('guy', 0.8029448986053467),
 ('lady', 0.760288655757904),
 ('girl', 0.7514596581459045),
 ('gentleman', 0.7495577335357666),
 ('boy', 0.7465522289276123),
 ('soldier', 0.7274065017700195),
 ('kid', 0.675993800163269),
 ('poet', 0.6630818247795105),
 ('rabbi', 0.6589103937149048)]

In [ ]:
# man - boy + girl 
model.wv.most_similar(positive=['man', 'girl'], negative=['boy'], topn=3) 
#==> woman이 나오길 원해요................

[('woman', 0.8275146484375),
 ('guy', 0.7304987907409668),
 ('lady', 0.719184398651123)]

### 힘들게 만든 Word2Vec 모델 저장하고 로드하기

In [ ]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v') # wv모델저장

In [ ]:
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드
model_result = loaded_model.most_similar("man") # 여기서는 wv가 빠져있다!!!!!!!!!!!!!주의!!!!!!!!!
model_result #저장된모델로 같은결과나오는지확인

[('woman', 0.8661956787109375),
 ('guy', 0.8029448986053467),
 ('lady', 0.760288655757904),
 ('girl', 0.7514596581459045),
 ('gentleman', 0.7495577335357666),
 ('boy', 0.7465522289276123),
 ('soldier', 0.7274065017700195),
 ('kid', 0.675993800163269),
 ('poet', 0.6630818247795105),
 ('rabbi', 0.6589103937149048)]

In [ ]:
!ls -l

total 97944
-rw-r--r-- 1 root root 25751709 Feb 24 01:09 eng_w2v
drwxr-xr-x 1 root root     4096 Feb 22 14:38 sample_data
-rw-r--r-- 1 root root 74533638 Feb 24 00:54 ted_en-20160408.xml
